![IFMG](https://images.even3.com.br/sBWZnWPFUBgLOGciSZc4G5ZQy7Q=/1100x440/smart/even3.blob.core.windows.net/banner/ARTEPARASITE3.993b3db9f908426e9833.png)

---

# Introdução ao Tensorflow - Parte 5

---

#### Professor: Felipe Reis

#### Data: 20-10-2021

---
### Importação de bibliotecas 

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import time

#import plot_images
from xject_helper import xject_helper as helper

#tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#código para compatibilidade de redes convolucionais com algumas placas de vídeo (ex.: série 16*)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

---
## Base de Dados CIFAR-10

A base de dados CIFAR-10 consiste em 60.000 imagens coloridas de 32x32, divididas em 10 classes.
Cada classe contém 6.000 imagens por classe. 
O conjunto é dividido em 50.000 imagens de treinamento e 10.000 imagens de teste.

CIFAR é o acrônimo de Canadian Institute For Advanced Research, proprietário da base de dados.
O CIFAR-10 e CIFAR-100 são subconjuntos rotulados de uma base de dados de pequenas imagens que contém 80 milhões de amostras. 

As imagens foram coletadas por Alex Krizhevsky, Vinod Nair e Geoffrey Hinton.

A base está disponível em: https://www.cs.toronto.edu/~kriz/cifar.html

Ela também está disponível na página do Tensorflow: https://www.tensorflow.org/datasets/catalog/cifar10

![CIFAR 10](https://cdn-images-1.medium.com/max/1200/1*SZnidBt7CQ4Xqcag6rd8Ew.png)

Fonte: *Jannik Zürn. **Training a CIFAR-10 classifier in the cloud using TensorFlow and Google Colab**. 2018. Disponível em: https://jannik-zuern.medium.com/training-a-cifar-10-classifier-in-the-cloud-using-tensorflow-and-google-colab-f3a5fbdfe24d. Acesso em: 05 de outubro de 2021.*

In [ ]:
#download da base de dados
cifar10 = keras.datasets.cifar10

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#remove 1 dimensão dos arrays de label
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

In [ ]:
#impressão da quantidade de imagens da base
print('Conjunto treino: {} amostras'.format(len(train_images)))
print('Conjunto teste: {} amostras'.format(len(test_images)))

In [ ]:
#definicao dos nomes das classes
class_names = ['Avião', 'Automóvel', 'Pássaro', 'Gato', 'Veado', 
               'Cachorro', 'Sapo', 'Cavalo', 'Barco', 'Caminhão']

### Análise dos Dados

In [ ]:
helper.plot_images(train_images, train_labels, class_names, random=False)

---
## Pré processamento

Podemos classificar o conjunto de dados utilizando os mesmos modelos utilizados no Notebook 1.

In [ ]:
#pré processamento
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
helper.plot_images(train_images, train_labels, class_names, random=False)

---
## Redes Convolucionais

A rede neural abaixo é uma rede convolucional simples, contendo apenas uma camada convolucional.

Sua arquitetura é baseada no modelo existente no Notebook 1, com adição de uma cada de convolução.


#### Camada Convolucional

A camada convolucional executa um convolução de em duas dimensões (ex.: convolução espacial sobre imagens).

Essa camada cria um *kernel* de convolução que realiza a operação de convolução na entrada da camada para produzir um tensor de saídas. 

Para usá-la é necessário utilizar o comando `keras.layers.Conv2D`.

Mais informações em: https://keras.io/api/layers/convolution_layers/convolution2d/

In [ ]:
def load_model_1():
    return keras.Sequential([
        #convolução com 32 nós
        keras.layers.Conv2D(32, 3, activation='relu'),

        #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
        keras.layers.Flatten(input_shape=(28, 28)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

---
## Compilação do Modelo

Informações detalhadas sobre os métodos de otimização, perda e métricas serão exibidas nos próximos notebooks e nas parte teórica.

In [ ]:
#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

### Principais Parâmetros 

As camadas convolucionais possuem diversos parâmetros, que podem ser utilizados para ajuste do modelo.

Os parâmetros principais (ou mais utilizados) estão descritos abaixo.

*obs.: A alteração dos parâmetros pode não ser trivial, podendo requerer adequação das camadas seguintes. O cálculo de saída de uma camada convolucional não será descrito neste documento.*

Mais informações em: https://keras.io/api/layers/convolution_layers/convolution2d/

#### *Filters*

Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
Números de filtros de saída da convolução.

Pode ser associada à profundidade em um modelo convolucional.

![filters](https://cs231n.github.io/assets/cnn/cnn.jpeg)

Fonte: *Li, F.-F., Krishna, R., and Xu, D. **Convolutional neural networks (cnns / convnets)**. 2021. Disponível em: https://cs231n.github.io/convolutional-networks/. Acesso em: 26 de Janeiro de 2021.*

#### Kernel Size

Tamanho do *kernel* que desliza sobre a imagem durante o processo de convolução.

São definidos por um tupla de 2 inteiros, correspondente ao largura e altura do filtro. 

![kernel](https://miro.medium.com/max/667/1*naUX8KsDblRJebhrqERtBw.png)

Fonte: *Yunus, M. **11 Artificial Neural Network (ANN) — Part 6 Konsep Dasar Convolutional Neural Network (CNN).**. 2020. Disponível em: https://yunusmuhammad007.medium.com/11-artificial-neural-network-ann-part-6-konsep-dasar-convolutional-neural-network-cnn-3cc10fd9cf69 . Acesso em: 26 de Janeiro de 2021.*

#### *Strides*

Corresponde à taxa de "deslizamento" dos *kernels*, em pixels.

São definidos por um tupla de 2 inteiros, correspondente ao largura e altura do filtro. 

#### *Padding*

*Padding* ou *zero-padding* corresponde ao parâmetro correspondente à adição de zeros à margem da imagem para que a saída de uma convolução tenha um tamanho (largura e altura) desejado.

O parâmetros pode receber valores como "*valid*" ou "*same*", de modo a gerenciar a saída de dados. 

![padding](https://miro.medium.com/max/667/1*Ch4NraXzYjdZrSoA9sGZTg.png)

Fonte: *Yunus, M. **11 Artificial Neural Network (ANN) — Part 6 Konsep Dasar Convolutional Neural Network (CNN).**. 2020. Disponível em: https://yunusmuhammad007.medium.com/11-artificial-neural-network-ann-part-6-konsep-dasar-convolutional-neural-network-cnn-3cc10fd9cf69 . Acesso em: 26 de Janeiro de 2021.*

#### Alteração da quantidade de filtros

Podemos gerar um novo modelo, com alteração da quantidade de filtros.

In [ ]:
def load_model_2():
    return keras.Sequential([
        #convolução com 32 nós
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid',
                            input_shape=(32,32,3), activation='relu'),
        
        #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
        keras.layers.Flatten(input_shape=(28, 28)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#gera modelo
model2 = load_model_2() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model2.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model2.fit(train_images, train_labels, epochs=10)

#### Alteração do parâmetro *kernel_size*

Podemos gerar um novo modelo, com alteração do parâmetro *kernel_size*.

In [ ]:
def load_model_3():
    return keras.Sequential([
        #convolução com 32 nós
        keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding='valid',
                            input_shape=(32,32,3), activation='relu'),
        
        #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
        keras.layers.Flatten(input_shape=(28, 28)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#gera modelo
model3 = load_model_3() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model3.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model3.fit(train_images, train_labels, epochs=10)

#### Alteração do parâmetro *strides*

Podemos gerar um novo modelo, com alteração do parâmetro *stride*.

In [ ]:
def load_model_4():
    return keras.Sequential([
        #convolução com 32 nós
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(3,3), padding='valid',
                            input_shape=(32,32,3), activation='relu'),
        
        #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
        keras.layers.Flatten(input_shape=(28, 28)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#gera modelo
model4 = load_model_4() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model4.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model4.fit(train_images, train_labels, epochs=10)

## Pooling

A função de pooling é utilizada para prover informação estatística a respeito das saídas próximas.

Tem como objetivo maximizar as vantagens da saída e tornar-se invariante a pequenos ruídos.

Podem também ser utilizadas para redução da quantidade de neurônios entre camadas da rede, agrupando resultados na
camada seguinte.

O método de *pooling* mais utilizado é o MaxPooling (Pooling Máximo). Para usá-lo é necessário utilizar o comando `keras.layers.MaxPooling2D`.

Mais informações: https://keras.io/api/layers/pooling_layers/

![Pooling](https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png)

Fonte: *Computer Science Wiki. **Max-pooling / Pooling**. 2020. Disponível em: https://computersciencewiki.org/index.php/Max-pooling_/_Pooling. Acesso em: 08 de Setembro de 2020.* 


In [ ]:
def load_model_5():
    return keras.Sequential([
        #convolução com 32 nós
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(3,3), padding='valid',
                            input_shape=(32,32,3), activation='relu'),
        
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        
        #transforma a image em um array de imagens de duas dimensões (of 28 by 28 pixels)
        keras.layers.Flatten(input_shape=(28, 28)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#gera modelo
model5 = load_model_5() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model5.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model5.fit(train_images, train_labels, epochs=10)

---
## Aumento Artificial de Dados (*Data Augmentation*)

Podemos diminuir o sobretreinamento (*overfitting*), por meio da criação de registros artificiais. Esse processo é conhecido como *data augmentation*.

O aumento artificial de dados dificulta o reconhecimento de amostras ("rede decora amostras") e favorece a capacidade de generalização.

Nas redes convolucionais previamente estudadas, pode ser percebido a existência de *overfitting*, quando dividimos os conjuntos em treinamento e validação.

Observa-se um crescimento da acurácia no conjunto de treinamento e a manutenção do desempenho no conjunto de validação.

In [ ]:
#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, validation_split=0.2, epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy', 'val_accuracy'], legend=['Treinamento', 'Validação'])

### Data Augmentation

Para aumento dos dados, é possível utilizar diferentes recursos computacionais, como bibliotecas e frameworks específicos para esse propósito.

O Tensorflow é capaz de gerar dados de diferentes formas. Por questões didáticas, optaremos por um gerador de imagem simples, provido pela interface do Keras.

Esse gerador é denominado `ImageDataGenerator`.

Ele é capaz de fazer diversas transformações nas imagens, como espelhamento, rotação, normalização da imagem, deslocamento de *pixels*, entre outros.

Outras funcionalidades também pode acrescentar opções como *blur*, aumento de saturação, adição de ruídos, aplicação de filtros, entre outros.

![Data augmentation](https://algorithmia.com/blog/wp-content/uploads/2018/08/word-image-8.png)

Fonte: *Algorithmia. **Introduction to Dataset Augmentation and Expansion**. 2018. Disponível em: https://algorithmia.com/blog/introduction-to-dataset-augmentation-and-expansion . Acesso em: 17 de outubro de 2021.*

Mais informações: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

#### ImageDataGenerator

Para usar o `ImageDataGenerator`, precisamos criar um gerador de imagem e definir quais as transformações iremos realizar.

Quanto mais transformações utilizarmos, mais amostras iremos processar (consequentemente mais tempo a rede irá demorar para treinar).

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,             #Normaliza a imagem de modo que ela tenha média zero
    featurewise_std_normalization=True,  #Normaliza a entrada para média de cores dos pixel da base de dados
    rotation_range=20,                   #intervalo de rotação
    width_shift_range=0.2,               #intervalo de deslocamento de pixels na largura
    height_shift_range=0.2,              #intervalo de deslocamento de pixels na altura
    horizontal_flip=True,                #espelhamento horizontal
    validation_split=0.2)

In [ ]:
#gera modelo
model1 = load_model_1() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(datagen.flow(train_images, train_labels, batch_size=32, subset='training'), 
           validation_data=datagen.flow(train_images, train_labels, batch_size=8, subset='validation'),
           epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy', 'val_accuracy'], legend=['Treinamento', 'Validação'])

----
----

## Tarefas 

### Tarefa 1 (20 minutos)

Treine a rede convolucional abaixo no conjunto de dados CIFAR-10.

#### Atividades Sugeridas

- Inclua rotinas de aumento artificial de dados;
- Varie as taxas de aprendizado e decaimento da rede;
- Plote gráficos para verificar a existência de overfitting.

In [ ]:
def load_model():
    return keras.Sequential([
        keras.layers.Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64, (5,5), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(3,3)),
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.Flatten(), 
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui

----

### Tarefa 2 (30 minutos)

Treine a rede convolucional abaixo no conjunto de dados Flowers.

https://www.tensorflow.org/datasets/catalog/tf_flowers

#### Atividades Sugeridas

- Inclua rotinas de aumento artificial de dados;
- Varie as taxas de aprendizado e decaimento da rede;
- Plote gráficos para verificar a existência de overfitting.

In [ ]:
def load_model():
    return keras.Sequential([
        keras.layers.Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64, (5,5), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(3,3)),
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.Flatten(), 
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui

----

### Tarefa 3 (60 minutos)

Treine a rede convolucional no conjunto de dados Stanford Dogs.

https://www.tensorflow.org/datasets/catalog/stanford_dogs

#### Atividades Sugeridas

- Inclua rotinas de aumento artificial de dados;
- Varie as taxas de aprendizado e decaimento da rede;
- Plote gráficos para verificar a existência de overfitting.

In [ ]:
def load_model():
    return keras.Sequential([
        keras.layers.Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64, (5,5), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(3,3)),
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.Flatten(), 
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui